In [2]:
import requests # για να φέρνουμε τον html κώδικα των ιστοσελίδων
from bs4 import BeautifulSoup # για να βρίσκουμε μέσα στον html κώδικα, τα περιεχόμενα που μας ενδιαφέρουν
import pandas as pd # για να βάζουμε τα δεδομένα που συλλέγουμε σε dataframes

import time # για να κάνουμε παύσεις ανάμεσα στα αιτήματα που κάνουμε στον server του site ώστε να μην τον ρίξουμε ή να μην μας αποκλείσει
import random # για να δημιουργούμε ποικιλία στον χρόνο των παύσεων ώστε τα αιτήματα στον server να μοιάζουν με ανθρώπινα και όχι μηχανικά

In [3]:
def get_webpage_soup(url):
  # Ορισμός των headers για το requests
  headers = {
        "User-Agent": "Mozilla/5.0"
    }
  # Αίτημα στο server να φέρει τον html κώδικα από το url
  response = requests.get(url, headers=headers)

  # Έλεγχος της κατάστασης της σελίδας
  if response.status_code != 200:
    print(f"Error: {response.status_code}")
    return None
  else:
    # Μετατροπή του πηγαίου κώδικα της ιστοσελίδας σε αναζητήσιμο κείμενο (soup)
    print(f"{url} was scraped successfully")
    return BeautifulSoup(response.text, 'html.parser')

In [4]:
def get_teaser_elements_from_articlesL(articlesL):
  articles_dataL = [] # κενή λίστα, στην οποία θα κάνουμε append() το dictionary με τα data κάθε άρθρου
  for article in articlesL:
    try:
      title = article.find("h3").text.strip()
    except:
      title=""
    try:
      article_url = article.find("a", {"class": "is-block hover-underline"})["href"]
    except:
      article_url=""
    try:
      date_str = article.date.text.strip()
      date_partsL = date_str.split(".")
      date_str = f"20{date_partsL[2]}-{date_partsL[1]}-{date_partsL[0]}"
      time_str = article.time.text.strip()
    except:
      date_str = ""
      time_str = ""
    try:
      author = article.find("div", {"class":"author-c"}).text.strip()
    except:
      author = ""
    try:
      section = article.find("span", {"class":"post-category"}).text.strip()
    except:
      section = ""
    try:
      excerpt = article.find("p", {"class":"post-description"}).text.strip()
    except:
      excerpt = ""
    # Δημιουργία λεξικού με τα data του άρθρου
    articleD = {"title": title, "article_url": article_url, "date": date_str, "time": time_str, "author": author, "section": section, "excerpt": excerpt}

    # Προσθήκη των data του κάθε άρθρου στη λίστα των data των άρθρων
    articles_dataL.append(articleD)

  return articles_dataL

In [5]:
def scrape_in_teaser_pages(search_term, start_page_nbr, end_page_nbr):
  # Κενή αρχική λίστα
  all_articles_dataL = []

  print(f"\rScraping started", end="")

  for page_nbr in range(start_page_nbr, end_page_nbr+1): # βάζω +1 για να σκραπάρω και τον αριθμό του τελευταίου page που έχω ορίσει
    # η δομή των teasers pages urls του kathmerini.gr
    url = f"https://www.in.gr/search/{search_term}/page/{page_nbr}/"

    print(f"\rScraping page {page_nbr}: {url}" + " "*2, end="")

    # φέρνω τη σούπα της σελίδας με το function που έφτιαξα παραπάνω στο βήμα 2
    soup = get_webpage_soup(url)

    # βάζω τον κώδικα με τον οποίο βρίσκω όλα τα blocks των άρθρων - το έφτιαξα στο βήμα 6
    articlesL = soup.find_all("article")

    # εξορύσσω τα δεδομένα όλων των άρθρων της σελίδας με το function που έφτιαξα στο βήμα 8
    page_articles_dataL = get_teaser_elements_from_articlesL(articlesL)

    # προσθέτω τα δεδομένα των άρθρων της σελίδας στη λίστα με τα δεδομένα όλων των άρθρων - τη δημιούργησα στην αρχή αυτού του function
    all_articles_dataL.extend(page_articles_dataL)

    # Τυχαία καθυστέρηση μεταξύ 1 και 3 δευτερολέπτων ανάμεσα στα page scrapes
    delay = random.uniform(1, 3)
    time.sleep(delay)

  # Δημιουργία dataframe από τη λίστα με τα λεξικά των δεδομένων κάθε άρθρου
  df = pd.DataFrame(all_articles_dataL)
  print(f"\rScraping completed. Fetshed {len(df)} articles.")

  return df

**Δοκιμή στην σελίδα της αναζήτησης για τον όρο `Τραμπ`**

In [6]:
in_tempi_teasers_df = scrape_in_teaser_pages("τεμπη", 9, 24)

Scraping page 9: https://www.in.gr/search/τεμπη/page/9/  https://www.in.gr/search/τεμπη/page/9/ was scraped successfully
Scraping page 10: https://www.in.gr/search/τεμπη/page/10/  https://www.in.gr/search/τεμπη/page/10/ was scraped successfully
Scraping page 11: https://www.in.gr/search/τεμπη/page/11/  https://www.in.gr/search/τεμπη/page/11/ was scraped successfully
Scraping page 12: https://www.in.gr/search/τεμπη/page/12/  https://www.in.gr/search/τεμπη/page/12/ was scraped successfully
Scraping page 13: https://www.in.gr/search/τεμπη/page/13/  https://www.in.gr/search/τεμπη/page/13/ was scraped successfully
Scraping page 14: https://www.in.gr/search/τεμπη/page/14/  https://www.in.gr/search/τεμπη/page/14/ was scraped successfully
Scraping page 15: https://www.in.gr/search/τεμπη/page/15/  https://www.in.gr/search/τεμπη/page/15/ was scraped successfully
Scraping page 16: https://www.in.gr/search/τεμπη/page/16/  https://www.in.gr/search/τεμπη/page/16/ was scraped successfully
Scraping pa

In [7]:
in_tempi_teasers_df

title  \
0    Τέμπη: Νέα αποκάλυψη για τον ρόλο Τριαντόπουλο...   
1    Η πληγή που άνοιξαν στην κυβέρνηση τα Τέμπη εί...   
2    Μαζικές προσαγωγές μετά το νέο συλλαλητήριο γι...   
3    Τι σημαίνουν τα υψωμένα χέρια; - Από το Black ...   
4    Ανδρουλάκης: Έχετε προσωπικές ευθύνες για τα Τ...   
..                                                 ...   
395  Καταδικάζουν την κυβέρνηση «που έπνιξε στα χημ...   
396              Τέμπη: συγκέντρωση γεμάτη αξιοπρέπεια   
397  Πανό για τα Τέμπη από τους οπαδούς του Ολυμπια...   
398  Ο Φάμελλος θα επιδιώξει συνεννόηση με τα προοδ...   
399  Κουτσούμπας από Σέρρες: Να αποκαλυφθούν οι υπε...   

                                           article_url        date   time  \
0    https://www.in.gr/2025/03/08/politics/politiki...  2025-03-08  17:27   
1    https://www.in.gr/2025/03/08/politics/in-confi...  2025-03-08  09:00   
2    https://www.in.gr/2025/03/08/greece/mazikes-pr...  2025-03-08  08:30   
3    https://www.in.gr/2025/03/08/greece/ypsomena-x...  2025-03-08  07:38   
4    https://www.in.gr/2025/03/07/politics/politiki...  2025-03-07  21:49   
..                                                 ...         ...    ...   
395  https://www.in.gr/2025/01/26/greece/tempi-kata...  2025-01-26  21:41   
396  https://www.in.gr/2025/01/26/politics/politiki...  2025-01-26  21:00   
397  https://www.in.gr/2025/01/26/sports/football/p...  2025-01-26  20:55   
398  https://www.in.gr/2025/01/26/politics/politiki...  2025-01-26  19:28   
399  https://www.in.gr/2025/01/26/politics/politiki...  2025-01-26  18:44   

                author                 section  \
0              Σύνταξη                Κατάθεση   
1     Τζέλα Δελαφράγκα         in Confidential   
2              Σύνταξη                  Ελλάδα   
3    Αφροδίτη Τζιαντζή    Παγκόσμια χειρονομία   
4              Σύνταξη  «Τρομακτικός κυνισμός»   
..                 ...                     ...   
395            Σύνταξη     «Αδικαιολόγητη βία»   
396        Ολγα Στέφου           Χωρίς οξυγόνο   
397            Σύνταξη              Ποδόσφαιρο   
398            Σύνταξη                  ΣΥΡΙΖΑ   
399            Σύνταξη                     ΚΚΕ   

                                               excerpt  
0    ΜΙα σημαντική μαρτυρία για την παρουσία του Χρ...  
1    Όσο κι αν ο πρωθυπουργός και η κυβέρνησή του ξ...  
2    Μαθητές και φοιτητές πραγματοποίησαν μαζική πο...  
3    Τα υψωμένα χέρια ειρηνικών διαδηλωτών μπροστά ...  
4    «Σήμερα αναζητούμε ακόμη ποιο ήταν το άγνωστο ...  
..                                                 ...  
395  «Η κυβέρνηση επέλεξε να πνίξει στα χημικά τη σ...  
396  Αν σήμερα γράφτηκε ιστορία, αυτό θα το κρίνει ...  
397  Οι οπαδοί του Ολυμπιακού σήκωσαν πανό για τα Τ...  
398  Ο πρόεδρος του ΣΥΡΙΖΑ - ΠΣ Σωκράτης Φάμελλος θ...  
399  Ο Δημήτρης Κουτσούμπας παρευρέθηκε στη συγκέντ...  

[400 rows x 7 columns]

In [8]:
in_tempi_teasers_df.to_csv('/content/drive/MyDrive/in_tempi_teasers.csv', index=False)

OSError: Cannot save file into a non-existent directory: '/content/drive/MyDrive'

In [ ]:
in_tempi_teasers_df.to_csv('in_tempi_teasers.csv', index=False)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#from google.colab import files
#files.download('in_tempi_teasers.csv')

articles scraper

In [ ]:
import requests # για να φέρνουμε τον html κώδικα των ιστοσελίδων
from bs4 import BeautifulSoup # για να βρίσκουμε μέσα στον html κώδικα, τα περιεχόμενα που μας ενδιαφέρουν
import pandas as pd # για να βάζουμε τα δεδομένα που συλλέγουμε σε dataframes

import time # για να κάνουμε παύσεις ανάμεσα στα αιτήματα που κάνουμε στον server του site ώστε να μην τον ρίξουμε ή να μην μας αποκλείσει
import random # για να δημιουργούμε ποικιλία στον χρόνο των παύσεων ώστε τα αιτήματα στον server να μοιάζουν με ανθρώπινα και όχι μηχανικά

from tqdm import tqdm # για να βάλω progress bar με το οποίο θα παρακολουθώ την πορεία του scraping

In [ ]:
def get_webpage_soup(url):
  # Ορισμός των headers για το requests
  headers = {
        "User-Agent": "Mozilla/5.0"
    }
  # Αίτημα στο server να φέρει τον html κώδικα από το url
  response = requests.get(url, headers=headers)

  # Έλεγχος της κατάστασης της σελίδας
  if response.status_code != 200:
    #print(f"Error: {response.status_code}")
    return None
  else:
    # Μετατροπή του πηγαίου κώδικα της ιστοσελίδας σε αναζητήσιμο κείμενο (soup)
    # print(f"{url} was scraped successfully")
    return BeautifulSoup(response.text, 'html.parser')

In [ ]:
in_tempi_teasers_df = pd.read_csv("/content/in_tempi_teasers.csv")
in_tempi_teasers_df.head()

In [ ]:
article_url = in_tempi_teasers_df.loc[0, "article_url"]
article_url

In [ ]:
soup = get_webpage_soup(article_url)

In [ ]:
article_block = soup.find("article")
print(article_block.prettify())

In [ ]:
#Εντοπισμός των tags/classes των elements που θέλω να μαζέψω

In [ ]:
section = soup.find("a", class_="post-category").text.strip()
section

In [ ]:
title = soup.find("h1").text.strip()
title

In [ ]:
excerpt = soup.find("h2" ,class_="is-sans-serif-font is-medium is-darkblue article-summary").text.strip()
excerpt = excerpt.replace("\xa0", "").strip()
excerpt

In [ ]:
feat_img_link = soup.find("figure", class_="thumb-holder").find("img")["src"]
feat_img_link

In [ ]:
image_caption = soup.find("figure", class_="thumb-holder").find("img")["alt"]
image_caption

In [ ]:
!pip install dateparser #εγκατασταση για να γινει η μετατροπη απο ελληνικη ημερομηνια σε αριθμο

In [ ]:
datetime_str = soup.find("span", class_="is-sans-serif-font")

In [ ]:
import dateparser


In [ ]:
tags = soup.find_all("span", class_="is-sans-serif-font")
for i, t in enumerate(tags):
    print(f"[{i}] → {t.text.strip()}")

In [ ]:
datetime_str = tags[28].text.strip()
datetime_str

In [ ]:
datetime_str = tags[28].text.strip()  # ✔️ τώρα έχεις σωστό string
datetime_str = datetime_str.replace('|', '').strip()  # ✔️ τώρα μπορείς να κάνεις replace
datetime_str

In [ ]:
import dateparser
datetime_str = datetime_str.replace('|', '').strip()
datetime_str = dateparser.parse(datetime_str, languages=['el'])
print(datetime_str)

In [ ]:
import datetime

In [ ]:
date_str = datetime_str.date()
date_str = str(date_obj)
date_str

In [ ]:
time_str = datetime_str.time()
time_str = str(time_str)
time_str

In [ ]:
# το κείμενο του άρθρου βρίσκεται μέσα σε <p> tags μέσα στο <div> με id "main-content"
text_body_p_tagsL = article_block.find("div", class_= "inner-main-article").find_all("p")
# από κάθε p_tag παίρνω το text, το κάνω strip() και το βάζω σε μια λίστα
text_body_textsL = []
for p_tag in text_body_p_tagsL:
  p_tag_text = p_tag.text.strip()
  text_body_textsL.append(p_tag_text)
# συνένωση των κειμένων όλων των κειμένων των <p> tags
text_body = " ".join(text_body_textsL)
text_body

In [ ]:
def get_article_elements_from_soup(soup, article_url):
  # απομόνωση του block με το περιεχόμενο του άρθρου απόο όλη τη σούπα της σελίδας
  article_block = soup.find("article")
  # article url
  article_url = article_url # το παίρνω από το arguement του function
  # title
  try:
    title = soup.find("h1").text.strip()
  except:
    title=""
  # datetime
  try:
    datetime_str = datetime_str = tags[28].text.strip()
    datetime_str = datetime_str.replace('|', '').strip()
    import dateparser
    datetime_str = datetime_str.replace('|', '').strip()
    datetime_str = dateparser.parse(datetime_str, languages=['el'])
    date_str = datetime_str.date()
    date_str = str(date_str)
    time_str = datetime_str.time()
    time_str = str(time_str)
  except:
    date_str = ""
    time_str = ""
  # section
  try:
    section = soup.find("a", class_="post-category").text.strip()
  except:
    section = ""
  # author
  try:
    author = soup.find("a", class_='blue-c vcard author').text.strip()
  except:
    author = ""
  # excerpt
  try:
   excerpt = soup.find("h2" ,class_="is-sans-serif-font is-medium is-darkblue article-summary").text.strip()
   excerpt = excerpt.replace("\xa0", "").strip()
  except:
    excerpt=""
  # text body
  try:
    text_body_p_tagsL = article_block.find("div", class_= "inner-main-article").find_all("p")
    text_body_textsL = []
    for p_tag in text_body_p_tagsL:
      p_tag_text = p_tag.text.strip()
      text_body_textsL.append(p_tag_text)
    text_body = " ".join(text_body_textsL)
    text_body
  except:
    text_body=""
  # feature image
  try:
    feat_img_link = soup.find("figure", class_="thumb-holder").find("img")["src"]
  except:
    feat_img_link=""
  # feature image caption
  try:
    image_caption = soup.find("figure", class_="thumb-holder").find("img")["alt"]
  except:
    image_caption=""

  # Δημιουργία λεξικού με τα data του άρθρου
  articleD = {"article_url": article_url, "title": title, "date": date_str, "time": time_str, "section": section, "author": author,
              "excerpt": excerpt, "text_body": text_body, "feat_img_link": feat_img_link, "feat_img_cation": image_caption}

  return articleD

In [ ]:
articleD = get_article_elements_from_soup(soup, article_url)
articleD

In [ ]:
def scrape_in_articles_pages(in_tempi_teasers_df):
  # δημιουργία κενής λίστας στην οποία θα προσθέτουμε ένα-ένα τα λεξικά με τα δεδομένα των άρθρων που θα σκραπάρουμε
  all_articles_dataL = []
  # λούπα σε κάθε row του teasers_df για να παίρνουμε ένα-ένα τα urls των άρθρων και να το σκραπάρουμε
  # χρησιμοποιούμε το tqdm για να βλέπουμε σε ένα progress bar πού βρισκόμαστε και πόσος χρόνος απομένει ακόμα
  for idx, row in tqdm(in_tempi_teasers_df.iterrows(), total=in_tempi_teasers_df.shape[0], desc="Processing rows"):
    # παίρνουμε το url του άρθρου που θα σκραπάρουμε από τη στήλη "article_url" του row του teasers_df στο οποίο βρίσκεται η for loop
    article_url = row['article_url']
    # scrape της σελίδας του url του κάθε άρθρου με το function που φτιάξαμε στο βήμα 2
    soup = get_webpage_soup(article_url)
    # δημιουργία του λεξικού με τα δεδομένα του κάθε άρθρου με το function που φτιάξαμε στο βήμα 9
    articleD = get_article_elements_from_soup(soup, article_url)
    # προσθήκη του λεξικού με τα δεδομένα του κάθε άρθρου στη λίστα που δημιουργήσαμε στη αρχή
    all_articles_dataL.append(articleD)
    # καθυστέρηση ανάμεσα στα requests στο server
    delay = random.uniform(1, 3)
    time.sleep(delay)
  # δημιουργία dataframe από τη λίστα των λεξικών με τα δεδομένα κάθε άρθρου
  df = pd.DataFrame(all_articles_dataL)
  print(f"\nScraping completed. Fetshed {len(df)} articles.")
  return df

In [ ]:
in_tempi_articles_df = scrape_in_articles_pages(in_tempi_teasers_df)

In [ ]:
in_tempi_articles_df

In [ ]:
def get_article_elements_from_soup(soup, article_url):
    import dateparser
    article_block = soup.find("article")

    try:
        title = soup.find("h1").text.strip()
    except:
        title = ""

    # datetime
    try:
        datetime_tag = soup.find_all("span", class_="is-sans-serif-font")
        for tag in datetime_tag:
            if any(month in tag.text for month in ["Ιανουαρίου", "Φεβρουαρίου", "Μαρτίου", "Απριλίου", "Μαΐου", "Ιουνίου",
                                                    "Ιουλίου", "Αυγούστου", "Σεπτεμβρίου", "Οκτωβρίου", "Νοεμβρίου", "Δεκεμβρίου"]):
                datetime_str = tag.text.strip()
                break
        datetime_str = datetime_str.replace('|', '').strip()
        datetime_obj = dateparser.parse(datetime_str, languages=['el'])
        date_str = str(datetime_obj.date())
        time_str = str(datetime_obj.time())
    except:
        date_str = ""
        time_str = ""

    try:
        section = soup.find("a", class_="post-category").text.strip()
    except:
        section = ""

    try:
        author = soup.find("a", class_='blue-c vcard author').text.strip()
    except:
        author = ""

    try:
        excerpt = soup.find("h2", class_="is-sans-serif-font is-medium is-darkblue article-summary").text.strip()
        excerpt = excerpt.replace("\xa0", "").strip()
    except:
        excerpt = ""

    try:
        text_body_p_tagsL = article_block.find("div", class_="inner-main-article").find_all("p")
        text_body_textsL = [p.text.strip() for p in text_body_p_tagsL]
        text_body = " ".join(text_body_textsL)
    except:
        text_body = ""

    try:
        feat_img_link = soup.find("figure", class_="thumb-holder").find("img")["src"]
    except:
        feat_img_link = ""

    try:
        image_caption = soup.find("figure", class_="thumb-holder").find("img")["alt"]
    except:
        image_caption = ""

    articleD = {
        "article_url": article_url,
        "title": title,
        "date": date_str,
        "time": time_str,
        "section": section,
        "author": author,
        "excerpt": excerpt,
        "text_body": text_body,
        "feat_img_link": feat_img_link,
        "feat_img_caption": image_caption,
    }

    return articleD


In [ ]:
def scrape_in_articles_pages(in_tempi_teasers_df):
  # δημιουργία κενής λίστας στην οποία θα προσθέτουμε ένα-ένα τα λεξικά με τα δεδομένα των άρθρων που θα σκραπάρουμε
  all_articles_dataL = []
  # λούπα σε κάθε row του teasers_df για να παίρνουμε ένα-ένα τα urls των άρθρων και να το σκραπάρουμε
  # χρησιμοποιούμε το tqdm για να βλέπουμε σε ένα progress bar πού βρισκόμαστε και πόσος χρόνος απομένει ακόμα
  for idx, row in tqdm(in_tempi_teasers_df.iterrows(), total=in_tempi_teasers_df.shape[0], desc="Processing rows"):
    # παίρνουμε το url του άρθρου που θα σκραπάρουμε από τη στήλη "article_url" του row του teasers_df στο οποίο βρίσκεται η for loop
    article_url = row['article_url']
    # scrape της σελίδας του url του κάθε άρθρου με το function που φτιάξαμε στο βήμα 2
    soup = get_webpage_soup(article_url)
    # δημιουργία του λεξικού με τα δεδομένα του κάθε άρθρου με το function που φτιάξαμε στο βήμα 9
    articleD = get_article_elements_from_soup(soup, article_url)
    # προσθήκη του λεξικού με τα δεδομένα του κάθε άρθρου στη λίστα που δημιουργήσαμε στη αρχή
    all_articles_dataL.append(articleD)
    # καθυστέρηση ανάμεσα στα requests στο server
    delay = random.uniform(1, 3)
    time.sleep(delay)
  # δημιουργία dataframe από τη λίστα των λεξικών με τα δεδομένα κάθε άρθρου
  df = pd.DataFrame(all_articles_dataL)
  print(f"\nScraping completed. Fetshed {len(df)} articles.")
  return df

In [ ]:
in_tempi_articles_df = scrape_in_articles_pages(in_tempi_teasers_df)

In [ ]:
in_tempi_articles_df

In [ ]:
in_tempi_articles_df.head(30)

In [ ]:
in_tempi_articles_df.to_csv('in_tempi_articles.csv', index=False)

In [ ]:
in_tempi_articles_df.to_csv('/content/drive/MyDrive/in_tempi_articles.csv', index=False)

In [ ]:
from google.colab import files
files.download('in_tempi_articles.csv')

In [ ]:
in_tempi_articles_df.loc[2].text_body